# Data Cleaning and Preparation for Transfer Analysis

This notebook outlines the complete process of cleaning raw football player statistics, handling missing values, standardizing data, and preparing datasets for transfer analysis. The goal is to make the data easily understandable and replicable.

---

## 1. Initial Setup and Data Loading

This section imports necessary libraries and defines utility functions for the initial cleaning steps, such as dropping columns with percentage values, extracting player names from URLs, and handling various data types and missing values.

### 1.1 Import Libraries

We start by importing all the required libraries for data manipulation and regular expressions.

In [20]:
import pandas as pd
import numpy as np
import os
import re
from functools import reduce

### 1.2 Define Data Cleaning Functions

These functions encapsulate specific cleaning logic, making the main processing loop cleaner and more readable.

#### `drop_columns_with_percent(df)`

This function identifies and removes columns that contain percentage signs, as these often represent calculated metrics that might not be suitable for direct comparison or aggregation.

In [21]:
def drop_columns_with_percent(df):
    """
    Drops columns from a pandas DataFrame if any of their values (as strings)
    contain a '%' sign. This is useful for removing calculated metrics that
    might not be directly comparable or are redundant.
    """
    columns_to_drop = [col for col in df.columns if df[col].astype(str).str.contains('%', na=False).any()]
    if columns_to_drop:
        print(f"Dropping columns containing '%': {', '.join(columns_to_drop)}")
        return df.drop(columns=columns_to_drop)
    else:
        print("No columns with '%' found to drop.")
        return df

#### `extract_and_format_name(url)`

This helper function extracts a player's name from a given URL string and formats it into a more readable title-case format.

In [22]:
def extract_and_format_name(url):
    """
    Extracts and formats a player's name from a URL.
    Example: '.../some-player-name' becomes 'Some Player Name'.
    Returns None if the URL is NaN or cannot be parsed.
    """
    if pd.isna(url):
        return None
    match = re.search(r'/([^/]+)$', url) # Regex to find string after the last '/'
    if match:
        # Capitalize each word in the extracted name (e.g., 'some-player-name' -> 'Some Player Name')
        return ' '.join(word.capitalize() for word in match.group(1).replace('_', '-').split('-'))
    return None

#### `process_url_column(df)`

This function applies the `extract_and_format_name` function to the 'URL' column and renames it to 'Player Name'. This is crucial for uniquely identifying players across different datasets.

In [23]:
def process_url_column(df):
    """
    Processes the 'URL' column to extract player names and renames the column to 'Player Name'.
    This standardizes player identification.
    """
    if 'URL' in df.columns:
        df = df.copy()
        df['URL'] = df['URL'].apply(extract_and_format_name)
        df = df.rename(columns={'URL': 'Player Name'})
        print("Processed 'URL' column to 'Player Name'.")
    else:
        print("'URL' column not found, skipping URL processing.")
    return df


#### `clean_and_impute(df)`

This function handles data type conversions and imputes missing values. It specifically addresses common issues like commas in numeric strings and fills specific columns with zeros, while imputing others with the median.

In [24]:
def clean_and_impute(df):
    """
    Performs data type conversion and imputation on the DataFrame.
    Specifically handles 'Minutes Played' and 'Possession - Touches' by converting them to numeric,
    imputes 'Defending - Penalties conceded' and 'Possession - Penalties awarded' with 0,
    and fills other numerical missing values with their respective column medians.
    """
    df = df.copy()

    # Convert columns that might contain commas as thousands separators to numeric
    columns_to_clean_numeric = ['Minutes Played', 'Possession - Touches']

    for col in columns_to_clean_numeric:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '', regex=False), errors='coerce')
            print(f"Converted '{col}' to numeric.")
        elif col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            print(f"'{col}' is already numeric.")
        else:
            print(f"'{col}' not found or not an object type, skipping numeric conversion.")


    # Impute specific columns with 0, as these often represent counts that were zero if not recorded.
    columns_to_impute_zero = ['Defending - Penalties conceded', 'Possession - Penalties awarded']
    for col in columns_to_impute_zero:
        if col in df.columns and df[col].isnull().sum() > 0:
            nan_count = df[col].isnull().sum()
            df[col].fillna(0, inplace=True)
            print(f"Imputed {nan_count} missing values in '{col}' with 0.")
        elif col in df.columns:
            print(f"'{col}' has no missing values.")
        else:
            print(f"'{col}' not found, skipping zero imputation.")

    # Impute remaining numerical columns with their median to preserve distribution shape and reduce outlier impact.
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]) and df[col].isnull().sum() > 0:
            nan_count = df[col].isnull().sum()
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"Imputed {nan_count} missing values in '{col}' with median ({median_val}).")

    return df

---

## 2. Processing Raw Datasets

This section iterates through the raw CSV files, applies the defined cleaning functions, and ensures all datasets have a consistent set of columns before saving them.

### 2.1 Define File Paths

List all the raw CSV files that need to be processed.

In [25]:
# List of CSV files to process
csv_file_paths = [
    '2018-19_Transfers_2016-17_to_2019-20_Stats.raw.csv',
    '2019-20_Transfers_2017-18_to_2020-21_Stats.raw.csv',
    '2020-21_Transfers_2018-19_to_2021-22_Stats.raw.csv',
    '2021-22_Transfers_2019-20_to_2022-23_Stats.raw.csv',
    '2022-23_Transfers_2020-21_to_2023-24_Stats.raw.csv',
]

processed_dfs = []

### 2.2 Loop Through Files and Apply Cleaning

Each file is read, cleaned using the defined functions, and then added to a list of processed DataFrames. Error handling is included to catch missing files or other processing issues.

In [26]:
for file_path in csv_file_paths:
    print(f"\n{'='*10} Processing file: {file_path} {'='*10}")
    try:
        df = pd.read_csv(file_path)
        print(f"Original shape: {df.shape}")

        df_cleaned = drop_columns_with_percent(df)
        df_cleaned = process_url_column(df_cleaned)
        df_cleaned = clean_and_impute(df_cleaned)

        processed_dfs.append(df_cleaned)
        print(f"Successfully processed {file_path}. Cleaned shape: {df_cleaned.shape}")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure it's in the correct directory.")
    except Exception as e:
        print(f"An unexpected error occurred while processing '{file_path}': {e}")


========== Processing file: 2018-19_Transfers_2016-17_to_2019-20_Stats.raw.csv ==========
Original shape: (84, 43)
Dropping columns containing '%': Defending - Aerial duels won %, Defending - Duels won %, Defending - Tackles won %, Passing - Cross accuracy, Passing - Long ball accuracy, Passing - Pass accuracy, Possession - Dribble success, Trait - Aerial duels won, Trait - Chances created, Trait - Defensive actions, Trait - Goals, Trait - Shot attempts, Trait - Touches
'URL' column not found, skipping URL processing.
Converted 'Minutes Played' to numeric.
'Possession - Touches' is already numeric.
Imputed 64 missing values in 'Defending - Penalties conceded' with 0.
Imputed 72 missing values in 'Possession - Penalties awarded' with 0.
Imputed 7 missing values in 'Defending - Aerial duels won' with median (1.36).
Imputed 9 missing values in 'Defending - Blocked' with median (0.29).
Imputed 4 missing values in 'Defending - Dribbled past' with median (0.815).
Imputed 3 missing values in

/tmp/ipython-input-24-1054012734.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0, inplace=True)
/tmp/ipython-input-24-1054012734.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

Original shape: (167, 48)
Dropping columns containing '%': Defending - Aerial duels won %, Defending - Duels won %, Defending - Tackles won %, Passing - Cross accuracy, Passing - Long ball accuracy, Passing - Pass accuracy, Possession - Dribble success, ShootingRank - xG on target (xGOT), Trait - Aerial duels won, Trait - Chances created, Trait - Defensive actions, Trait - Goals, Trait - Shot attempts, Trait - Touches
Processed 'URL' column to 'Player Name'.
Converted 'Minutes Played' to numeric.
'Possession - Touches' is already numeric.
Imputed 144 missing values in 'Defending - Penalties conceded' with 0.
Imputed 129 missing values in 'Possession - Penalties awarded' with 0.
Imputed 3 missing values in 'Defending - Aerial duels won' with median (1.35).
Imputed 17 missing values in 'Defending - Blocked' with median (0.365).
Imputed 6 missing values in 'Defending - Dribbled past' with median (0.79).
Imputed 5 missing values in 'Defending - Fouls committed' with median (1.08).
Imputed 

### 2.3 Identify Common Columns and Save Cleaned Data

After all files are individually cleaned, this step identifies the common columns across all processed DataFrames. This is crucial for creating a unified dataset where players' statistics can be directly compared. Each cleaned DataFrame is then filtered to include only these common columns and saved to a new CSV file.

In [27]:
if processed_dfs:
    # Find the common columns across all processed DataFrames
    # Using `reduce` with `intersection` ensures we get columns present in All dataframes.
    common_columns = list(reduce(lambda left, right: left.intersection(right), [df.columns for df in processed_dfs]))
    print(f"\nFound {len(common_columns)} common columns across all datasets.")
    print("Common columns found:")
    print(common_columns)

    # Filter each DataFrame to keep only the common columns and save
    for i, df in enumerate(processed_dfs):
        file_path = csv_file_paths[i]
        df_final = df[common_columns].copy()

        # Verify no missing values remain
        missing_after_all_steps = df_final.isnull().sum()
        if missing_after_all_steps.sum() == 0:
            print(f"\n✅ All missing values handled for {file_path}.")
        else:
            print(f"\n⚠️ Warning: Missing values still remain in {file_path}:")
            print(missing_after_all_steps[missing_after_all_steps > 0])

        # Define output file path
        base_name = os.path.splitext(file_path)[0]
        output_file_path = f"{base_name}_cleaned_common.csv"
        df_final.to_csv(output_file_path, index=False)
        print(f"Final DataFrame for {file_path} saved to: {output_file_path}")
else:
    print("\nNo dataframes were processed. Please check file paths and previous steps.")


Found 30 common columns across all datasets.
Common columns found:
['Player Name', 'Season', 'Average Rating', 'Defending - Aerial duels won', 'Defending - Blocked', 'Defending - Dribbled past', 'Defending - Duels won', 'Defending - Fouls committed', 'Defending - Interceptions', 'Defending - Penalties conceded', 'Defending - Possession won final 3rd', 'Defending - Recoveries', 'Defending - Tackles won', 'Discipline - Red cards', 'Discipline - Yellow cards', 'Minutes Played', 'Passing - Accurate long balls', 'Passing - Assists', 'Passing - Chances created', 'Passing - Successful crosses', 'Passing - Successful passes', 'Possession - Dispossessed', 'Possession - Fouls won', 'Possession - Penalties awarded', 'Possession - Successful dribbles', 'Possession - Touches', 'Possession - Touches in opposition box', 'ShootingRank - Goals', 'ShootingRank - Shots', 'ShootingRank - Shots on target']

✅ All missing values handled for 2018-19_Transfers_2016-17_to_2019-20_Stats.raw.csv.
Final DataFram

---

## 3. Merging Datasets into Pre and Post Transfer Aggregations

This section focuses on combining the cleaned individual season datasets into two main datasets: one representing player statistics *before* a transfer season and another representing statistics *after* a transfer season. This is crucial for analyzing the impact of transfers.

### 3.1 Define file paths for cleaned data

List the paths to the newly created cleaned and common column CSV files

In [28]:
all_cleaned_files = [
    '2018-19_Transfers_2016-17_to_2019-20_Stats.raw_cleaned_common.csv',
    '2019-20_Transfers_2017-18_to_2020-21_Stats.raw_cleaned_common.csv',
    '2020-21_Transfers_2018-19_to_2021-22_Stats.raw_cleaned_common.csv',
    '2021-22_Transfers_2019-20_to_2022-23_Stats.raw_cleaned_common.csv',
    '2022-23_Transfers_2020-21_to_2023-24_Stats.raw_cleaned_common.csv',
]

# Initialize empty lists to collect all pre and post transfer dataframes
pre_transfer_data = []
post_transfer_data = []

### 3.2 Helper Function: `season_to_numeric`

This function converts a season string (e.g., "2016/17") into its starting year (e.g., 2016) for easier comparison with transfer seasons.

In [29]:
def season_to_numeric(season_str):
    """
    Convert season string like '2016/17' to numeric value 2016 for comparison.
    Handles non-string inputs gracefully.
    """
    if isinstance(season_str, str) and '/' in season_str:
        try:
            return int(season_str.split('/')[0])
        except ValueError:
            return None
    return None

### 3.3 Process and Aggregate Each File

This loop iterates through each cleaned file, determines the transfer season, splits the data into pre and post-transfer periods based on the 'Season_Year' column, aggregates player stats by averaging them, and collects these aggregated dataframes.

In [30]:
for file in all_cleaned_files:
    print(f"\nProcessing aggregation for file: {file}")
    df = pd.read_csv(file)

    # Extract transfer season from filename (e.g., "2018-19_Transfers_...")
    filename = os.path.basename(file)
    match = re.match(r"(\d{4})-\d{2}_Transfers_", filename)
    if not match:
        print(f"Skipping {file}: Filename format unexpected.")
        continue

    transfer_season_str = match.group(1)
    transfer_year = int(transfer_season_str)

    print(f"Identified transfer year: {transfer_year}")

    # Convert 'Season' column in data to a numeric start year (e.g., "2016/17" -> 2016)
    df["Season_Year"] = df["Season"].apply(season_to_numeric)

    # Split into pre and post transfer datasets based on the transfer year
    pre_df = df[df["Season_Year"] < transfer_year]
    post_df = df[df["Season_Year"] >= transfer_year]

    # Aggregate stats per player by taking the mean of all numeric columns
    pre_agg = pre_df.groupby("Player Name").mean(numeric_only=True).round(2).reset_index()
    post_agg = post_df.groupby("Player Name").mean(numeric_only=True).round(2).reset_index()

    # Append the aggregated dataframes to their respective lists
    if not pre_agg.empty:
        pre_transfer_data.append(pre_agg)
        print(f"Added {len(pre_agg)} pre-transfer player aggregations.")
    else:
        print(f"No pre-transfer data found for {file}.")

    if not post_agg.empty:
        post_transfer_data.append(post_agg)
        print(f"Added {len(post_agg)} post-transfer player aggregations.")
    else:
        print(f"No post-transfer data found for {file}.")


Processing aggregation for file: 2018-19_Transfers_2016-17_to_2019-20_Stats.raw_cleaned_common.csv
Identified transfer year: 2018
Added 23 pre-transfer player aggregations.
Added 23 post-transfer player aggregations.

Processing aggregation for file: 2019-20_Transfers_2017-18_to_2020-21_Stats.raw_cleaned_common.csv
Identified transfer year: 2019
Added 31 pre-transfer player aggregations.
Added 31 post-transfer player aggregations.

Processing aggregation for file: 2020-21_Transfers_2018-19_to_2021-22_Stats.raw_cleaned_common.csv
Identified transfer year: 2020
Added 29 pre-transfer player aggregations.
Added 29 post-transfer player aggregations.

Processing aggregation for file: 2021-22_Transfers_2019-20_to_2022-23_Stats.raw_cleaned_common.csv
Identified transfer year: 2021
Added 21 pre-transfer player aggregations.
Added 21 post-transfer player aggregations.

Processing aggregation for file: 2022-23_Transfers_2020-21_to_2023-24_Stats.raw_cleaned_common.csv
Identified transfer year: 20

### 3.4 Combine All Aggregated Data and Save

Finally, all individual pre-transfer and post-transfer aggregated dataframes are combined. If a player appears in multiple aggregated dataframes (e.g., from different original files), their stats are further averaged to create a single, comprehensive pre-transfer and post-transfer dataset.

In [31]:
if pre_transfer_data:
    # Concatenate all pre-transfer data and then average stats for players appearing across multiple files
    pre_all = pd.concat(pre_transfer_data).groupby("Player Name").mean(numeric_only=True).round(2).reset_index()
    print(f"\nTotal unique players in pre-transfer dataset: {len(pre_all)}")
else:
    pre_all = pd.DataFrame() # Create empty DataFrame if no data
    print("\nNo pre-transfer data to combine.")


if post_transfer_data:
    # Concatenate all post-transfer data and then average stats for players appearing across multiple files
    post_all = pd.concat(post_transfer_data).groupby("Player Name").mean(numeric_only=True).round(2).reset_index()
    print(f"Total unique players in post-transfer dataset: {len(post_all)}")
else:
    post_all = pd.DataFrame() # Create empty DataFrame if no data
    print("No post-transfer data to combine.")

# Define folder path for saving outputs
folder_path = "./" # Current directory

# Save the combined results to CSV files
if not pre_all.empty:
    pre_all.to_csv(os.path.join(folder_path, "pre_transfer_dataset.csv"), index=False)
    print(f"Pre-transfer dataset saved to: {os.path.join(folder_path, 'pre_transfer_dataset.csv')}")
else:
    print("Pre-transfer dataset not saved as it's empty.")

if not post_all.empty:
    post_all.to_csv(os.path.join(folder_path, "post_transfer_dataset.csv"), index=False)
    print(f"Post-transfer dataset saved to: {os.path.join(folder_path, 'post_transfer_dataset.csv')}")
else:
    print("Post-transfer dataset not saved as it's empty.")


Total unique players in pre-transfer dataset: 148
Total unique players in post-transfer dataset: 148
Pre-transfer dataset saved to: ./pre_transfer_dataset.csv
Post-transfer dataset saved to: ./post_transfer_dataset.csv


---

## 4. Adding Player Position Information

This final section enriches your aggregated player statistics by adding a general player position category. This is crucial for further analysis, as player performance metrics can vary significantly by position. This section processes both `pre_transfer_dataset.csv` and `post_transfer_dataset.csv` to ensure consistent position mapping across both.

### 4.1 Define file paths and datasets to process

Here, we specify the input file containing player positions and define a list of dictionaries. Each dictionary indicates a statistics file (`stats_file`) and its corresponding desired output file name (`output_file`) after positions are merged.

In [33]:
# Define input file paths
positions_file = "transfer_dataset.cleaned.csv"

# Define a list of dictionaries, each containing input stats file and desired output file
datasets_to_process = [
    {
        "stats_file": "pre_transfer_dataset.csv", # Your pre-transfer stats file
        "output_file": "pre_transfer_with_general_positions.csv" # Adjusted name for clarity
    },
    {
        "stats_file": "post_transfer_dataset.csv", # Your post-transfer stats file
        "output_file": "post_transfer_with_general_positions.csv" # Adjusted name for clarity
    }
]

# Load the positions dataset once to avoid redundant reads
try:
    positions_df = pd.read_csv(positions_file)
    print(f"Loaded positions data from '{positions_file}' with shape: {positions_df.shape}")
except FileNotFoundError:
    print(f"Error: Positions file '{positions_file}' not found. Please ensure it's in the same directory as this notebook.")
    # Exit or raise an error as this file is crucial for the next steps
    raise FileNotFoundError(f"Required file '{positions_file}' not found.")
except Exception as e:
    print(f"An error occurred while loading '{positions_file}': {e}")
    raise

Loaded positions data from 'transfer_dataset.cleaned.csv' with shape: (238, 8)


### 4.2 Create Position Mapping Function

This function maps detailed player positions (e.g., 'centre-back', 'striker') to broader, more general categories (e.g., 'Defense', 'Attack'). This simplification can be very useful for higher-level analysis.

In [34]:
def map_position(position):
    """
    Maps a detailed player position to a general position group (Attack, Midfield, Defense).
    Returns None if the position does not match any known group.
    """
    if pd.isna(position):
        return None
    position = str(position).lower()
    if any(pos in position for pos in ['striker', 'forward', 'winger', 'attacker']):
        return 'Attack'
    elif any(pos in position for pos in ['midfielder', 'midfield']):
        return 'Midfield'
    elif any(pos in position for pos in ['defender', 'back', 'centre-back', 'full-back']):
        return 'Defense'
    else:
        return None

# Apply mapping to the 'Position' column in positions_df
print("Mapping detailed positions to general position groups in the positions dataset...")
positions_df['Position'] = positions_df['Position'].apply(map_position)
print("Position mapping complete.")

# Review some mapped positions for verification
print("\nSample of original and general positions from 'transfer_dataset.cleaned.csv':")
print(positions_df[['Player Name', 'Position', 'Position']].drop_duplicates().head(10))

Mapping detailed positions to general position groups in the positions dataset...
Position mapping complete.

Sample of original and general positions from 'transfer_dataset.cleaned.csv':
        Player Name  Position  Position
0    Ante Palaversa  Midfield  Midfield
1  Philippe Sandler   Defense   Defense
2        Ko Itakura   Defense   Defense
3     Daniel Arzani    Attack    Attack
4              Fred  Midfield  Midfield
5       Diogo Dalot   Defense   Defense
6        Ben Gibson   Defense   Defense
7          Angelino   Defense   Defense
8      Zack Steffen      None      None
9    Slobodan Tedic    Attack    Attack


### 4.3 Helper Function: Process and Save Each Dataset

This function encapsulates the logic for loading a statistics dataset, merging it with the pre-mapped position data, and saving the result. This makes the main loop cleaner and easier to manage.

In [35]:
def process_and_save_dataset(stats_filepath, output_filepath, positions_df_mapped):
    """
    Loads a stats dataset, merges it with mapped positions, and saves the result to the same directory.

    Args:
        stats_filepath (str): Path to the statistics CSV file.
        output_filepath (str): Desired name for the output CSV file.
        positions_df_mapped (pd.DataFrame): DataFrame containing 'Player Name' and 'Position'.
    """
    print(f"\nProcessing: {stats_filepath}")
    try:
        stats_df = pd.read_csv(stats_filepath)
        print(f"  Loaded stats data with shape: {stats_df.shape}")
    except FileNotFoundError:
        print(f"Error: Stats file '{stats_filepath}' not found. Skipping this dataset.")
        return
    except Exception as e:
        print(f"An error occurred while loading '{stats_filepath}': {e}")
        return

    # Merge with the main stats dataset using 'Player Name'
    print(f"  Merging with positions data...")
    merged_df = stats_df.merge(
        positions_df_mapped[['Player Name', 'Position']],
        on='Player Name',
        how='left'
    )

    # Check for missing positions after merge
    missing_positions_count = merged_df['Position'].isnull().sum()
    if missing_positions_count > 0:
        print(f"  ⚠️ Warning: {missing_positions_count} players in '{output_filepath}' have missing 'Position' after merge.")
        print("  This may indicate player name mismatches or unmapped positions in 'transfer_dataset.cleaned.csv'.")
    else:
        print(f"  ✅ All players in '{output_filepath}' successfully matched with a 'Position'.")

    print(f"  Merged DataFrame shape: {merged_df.shape}")

    # Step 4: Save the result to the current directory
    merged_df.to_csv(output_filepath, index=False)
    print(f"Successfully saved processed data to: {output_filepath}")

### 4.4 Process All Defined Datasets

This loop iterates through the `datasets_to_process` list and calls the helper function for each, ensuring both pre and post-transfer datasets are enriched with position information and saved.

In [36]:
# Process all datasets defined in the list using the helper function
for dataset_info in datasets_to_process:
    process_and_save_dataset(
        dataset_info["stats_file"],
        dataset_info["output_file"],
        positions_df
    )

print("\nAll specified datasets have been processed and saved with general position information.")
print("Data cleaning, merging, and position enrichment complete!")


Processing: pre_transfer_dataset.csv
  Loaded stats data with shape: (148, 30)
  Merging with positions data...
  ⚠️ Warning: 26 players in 'pre_transfer_with_general_positions.csv' have missing 'Position' after merge.
  This may indicate player name mismatches or unmapped positions in 'transfer_dataset.cleaned.csv'.
  Merged DataFrame shape: (148, 31)
Successfully saved processed data to: pre_transfer_with_general_positions.csv

Processing: post_transfer_dataset.csv
  Loaded stats data with shape: (148, 30)
  Merging with positions data...
  ⚠️ Warning: 26 players in 'post_transfer_with_general_positions.csv' have missing 'Position' after merge.
  This may indicate player name mismatches or unmapped positions in 'transfer_dataset.cleaned.csv'.
  Merged DataFrame shape: (148, 31)
Successfully saved processed data to: post_transfer_with_general_positions.csv

All specified datasets have been processed and saved with general position information.
Data cleaning, merging, and position enr